# Data Import From MINIO server

## A. Accessing MINIO Console

Step 1: Head to this link http://10.1.32.31:9001 on your browser

Step 2: Login using your username and password

## B. Create test bucket in your name and upload few files

<img style="width: 80%" src="minio-1.PNG">

<img src='minio-2.PNG'>

## C. Download data from MINIO to workspace

### 1. Import libraries

In [2]:
from minio import Minio
from dotenv import load_dotenv # add this line
import os

### 2. Create .env file with credentials 

#### Run these in the terminal

`echo "MINIO_ACCESS"=USERID >> .env`

`echo "MINIO_KEY"=PWD >> .env`

### 3. Load credentials securely

In [3]:
load_dotenv() # add this line
user = os.getenv('MINIO_ACCESS')
key = os.getenv('MINIO_KEY')

### 4. Initialize minio client

In [5]:
# Create client with access key and secret key with specific region.
client1 = Minio(
    "minio-0:9000",
    access_key=user,
    secret_key=key,
    secure=False
)

### 5. Load data into workspace

In [11]:
# Print buckets in data server
buckets = client1.list_buckets()
for b,bucket in enumerate(buckets):
    print(b, "Bucket: " ,bucket.name, bucket.creation_date)

0 Bucket:  elan 2022-03-11 04:19:28.004000+00:00


In [10]:
# Download your bucket
bucketname = 'elan'
os.makedirs(bucketname,exist_ok=True)
# List objects information.
objects = client1.list_objects(bucketname)
for o, obj in enumerate(objects):
    # Download data of an object.
    client1.fget_object(bucketname, obj.object_name, os.path.join(bucketname,obj.object_name))

## D. Upload data from workspace to new bucket in MINIO

### 1. Define function to delete bucket if it already exists

In [12]:
from minio.deleteobjects import DeleteObject
def remove_nonempty_bucket(client,bucket_name):
    if client.bucket_exists(bucket_name):
        if len(list(client.list_objects(bucket_name))) > 0:
            delete_object_list = map(lambda x: DeleteObject(x.object_name),
                                     client.list_objects(bucket_name,recursive=True),)
            errors = client.remove_objects(bucket_name, delete_object_list)
            for error in errors:
                print("error occured when deleting object", error)
        client.remove_bucket(bucket_name)
        print("Bucket ", bucket_name, " removed!")
    else:
        print("Bucket ", bucket_name, " does not exist!")

### 2. Set new bucket and upload folder

In [18]:
new_bucket = "dummy"
upload_folder = "elan/"

### 3. Delete bucket if it exists and create empty bucket

In [19]:
client = client1
buckets = client1.list_buckets()
bucketnames = [b.name for b in buckets]
if new_bucket in bucketnames:
    remove_nonempty_bucket(client,new_bucket)

client.make_bucket(new_bucket)

### 4. Upload Data to bucket

In [20]:
files = os.listdir(upload_folder)
for f in files:
    prefix = f[:2]
    result = client.fput_object(
    new_bucket, f, os.path.join(upload_folder,f),
    metadata={"prefix": prefix},
    )
    #print("created {0} object; etag: {1}, version-id: {2}".format(result.object_name, result.etag, result.version_id,),)
print("Upload Done")

Upload Done


### 5. Go to MINIO Console and check if data is uploaded

<img style="width: 80%" src="cap3.PNG">